# Importing

In [1]:
import numpy as np
import pandas as pd

# Loading Data

In [3]:
#Loading movies data and printing its shape
movies = pd.read_csv('D:\\Q3\\movies.xls',sep=';',encoding='latin-1').drop('Unnamed: 3',axis=1)
print('Shape of this dataset :',movies.shape)
movies.head(10)

Shape of this dataset : (3883, 3)


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [4]:
#Loading ratings data and printing its shape
ratings = pd.read_csv('D:\\Q3\\ratings.csv\\ratings.csv',sep=';')
print('Shape of this dataset :',ratings.shape)
ratings.head(10)

Shape of this dataset : (1000209, 4)


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [5]:
#Loading users data and printing its shape
users = pd.read_csv('D:\\Q3\\users.xls',sep=';')
print('Shape of this dataset :',users.shape)
users.head(10)

Shape of this dataset : (6040, 5)


,userId,gender,age,occupation,zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,6810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


# Recommendation System using Collaborative Filtering

In [6]:
rating_pivot = ratings.pivot_table(values='rating',columns='userId',index='movieId').fillna(0)
print('Shape of this pivot table :',rating_pivot.shape)
rating_pivot.head(10)

Shape of this pivot table : (3706, 6040)


userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Model training for Recommending movies based on users ratings.

In [7]:
#we will use k-NearestNeighbors algorithm to solve our problem
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(rating_pivot)

NearestNeighbors(metric='cosine')

# class of Collaborative filtering Recommendation Engine

In [8]:
class Recommender:
    def __init__(self): # This list will store movies that called atleast ones using recommend_on_movie method
        self.hist = [] 
        self.ishist = False # Check if history is empty
    
    # This method will recommend movies based on a movie that passed as the parameter
    def recommend_on_movie(self,movie,n_reccomend = 5):
        self.ishist = True
        movieid = int(movies[movies['title']==movie]['movieId'])
        self.hist.append(movieid)
        distance,neighbors = nn_algo.kneighbors([rating_pivot.loc[movieid]],n_neighbors=n_reccomend+1)
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in [movieid]]
        return recommeds[:n_reccomend]
    
    # This method will recommend movies based on history stored in self.hist list
    def recommend_on_history(self,n_reccomend = 5):
        if self.ishist == False:
            return print('No history found')
        history = np.array([list(rating_pivot.loc[mid]) for mid in self.hist])
        distance,neighbors = nn_algo.kneighbors([np.average(history,axis=0)],n_neighbors=n_reccomend + len(self.hist))
        movieids = [rating_pivot.iloc[i].name for i in neighbors[0]]
        recommeds = [str(movies[movies['movieId']==mid]['title']).split('\n')[0].split('  ')[-1] for mid in movieids if mid not in self.hist]
        return recommeds[:n_reccomend]

# Observation of this Recommendation Engine on some Movies

In [9]:
# linitializing the Recommender Object
recommender = Recommender()  


In [10]:
# Recommendation based on past watched movies, but the object just initialized. So, therefore no history found
recommender.recommend_on_history()

No history found


In [11]:
# Recommendation based on this movie 
recommender.recommend_on_movie('Home Alone 2')

['Home Alone (1990)',
 'Flintstones, The (1994)',
 'Richie Rich (1994)',
 'Home Alone 3 (1997)',
 'D2']

In [12]:
# Recommendation based on past watched movies and this time a movie is there in the history.
recommender.recommend_on_history()

['Home Alone (1990)',
 'Flintstones, The (1994)',
 'Richie Rich (1994)',
 'Home Alone 3 (1997)',
 'D2']

In [13]:
# Recommendation based on this movie
recommender.recommend_on_movie('Tigerland (2000)')

['Requiem for a Dream (2000)',
 'Yards, The (1999)',
 'Steal This Movie! (2000)',
 'Contender, The (2000)',
 'Dancer in the Dark (2000)']

In [14]:
# Recommendation based on past watched movies, and this time two movies is there in the history.
recommender.recommend_on_history()

['Home Alone (1990)',
 'Flintstones, The (1994)',
 'D2',
 'Richie Rich (1994)',
 'Mighty Ducks, The (1992)']

In [15]:
# Recommendation based on this movie
recommender.recommend_on_movie('Dracula')

['Spy Hard (1996)',
 'Bio-Dome (1996)',
 'Mafia! (1998)',
 'Gremlins 2',
 'Down Periscope (1996)']

In [16]:
# Recommendation based on past watched movies, and this time three movies is there in the history.
recommender.recommend_on_history()

['Home Alone (1990)',
 'Flintstones, The (1994)',
 'Richie Rich (1994)',
 'D2',
 'Robin Hood']

In [17]:
# Recommendation based on this movie
recommender.recommend_on_movie('Money Train (1995)')

['Specialist, The (1994)',
 'Bulletproof (1996)',
 'Marked for Death (1990)',
 'Beverly Hills Cop III (1994)',
 'Cliffhanger (1993)']

In [18]:
# Recommendation based on past watched movies, and this time four movies is there in the history.
recommender.recommend_on_history()

['Flintstones, The (1994)',
 'Home Alone (1990)',
 'Robin Hood',
 'Batman Forever (1995)',
 'Richie Rich (1994)']

In [19]:
# Recommendation based on this movie
recommender.recommend_on_movie('GoldenEye (1995)')

['Tomorrow Never Dies (1997)',
 'Die Hard 2 (1990)',
 'Rock, The (1996)',
 'Mission',
 'Clear and Present Danger (1994)']

In [20]:
# Recommendation based on past watched movies, and this time five movies is there in the history.
recommender.recommend_on_history()

['Tomorrow Never Dies (1997)',
 'Batman Returns (1992)',
 'Die Hard',
 'Die Hard 2 (1990)',
 'Rock, The (1996)']

# Recommendation System using Content Based Filtering

In [21]:
#Vectorization of contents of movies
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
genres = vectorizer.fit_transform(movies.genres).toarray()
contents = pd.DataFrame(genres,columns=vectorizer.get_feature_names())
print('Shape of the content table :',contents.shape)
contents.head(10)

Shape of the content table : (3883, 347)


C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,1919,1956,1963,1968,1974,1977,1978,1979,1980,1981,...,wight,willowbrook,witch,worrying,wrath,years,yellow,yes,york,yu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 # Model training for Recommending movies based on movies contents.

In [22]:
#we will use k-NearestNeighbors algorithm to solve our problem
from sklearn.neighbors import NearestNeighbors
nn_algo = NearestNeighbors(metric='cosine')
nn_algo.fit(contents)

NearestNeighbors(metric='cosine')

# class of Content Based filtering Recommendation Engine

In [23]:
class Recommender:
    def __init__(self): # This list will store movies that called atleast ones using recommend_on_movie method
        self.hist = [] 
        self.ishist = False # Check if history is empty
    
    # This method will recommend movies based on a movie that passed as the parameter
    def recommend_on_movie(self,movie,n_reccomend = 5):
        self.ishist = True
        iloc = movies[movies['title']==movie].index[0]
        self.hist.append(iloc)
        distance,neighbors = nn_algo.kneighbors([contents.iloc[iloc]],n_neighbors=n_reccomend+1)
        recommeds = [movies.iloc[i]['title'] for i in neighbors[0] if i not in [iloc]]
        return recommeds[:n_reccomend]
    
    # This method will recommend movies based on history stored in self.hist list
    def recommend_on_history(self,n_reccomend = 5):
        if self.ishist == False:
            return print('No history found')
        history = np.array([list(contents.iloc[iloc]) for iloc in self.hist])
        distance,neighbors = nn_algo.kneighbors([np.average(history,axis=0)],n_neighbors=n_reccomend + len(self.hist))
        recommeds = [movies.iloc[i]['title'] for i in neighbors[0] if i not in self.hist]
        return recommeds[:n_reccomend]

# Observation of this Recommendation Engine on some Movies

In [24]:
# linitializing the Recommender Object
recommender = Recommender()  

In [25]:
# Recommendation based on past watched movies, but the object just initialized. So, therefore no history found
recommender.recommend_on_history() 

No history found


In [26]:
# Recommendation based on this movie 
recommender.recommend_on_movie('Home Alone 2')

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Twin Peaks',
 'Amityville 1992',
 'Prom Night IV',
 'Tetsuo II',
 'Hellraiser III']

In [27]:
# Recommendation based on past watched movies, and this time a movie is there in the history.
recommender.recommend_on_history()

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Twin Peaks',
 'Amityville 1992',
 'Prom Night IV',
 'Tetsuo II',
 'Hellraiser III']

In [28]:
# Recommendation based on this movie
recommender.recommend_on_movie('Tigerland (2000)')

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Breaking the Waves (1996)',
 'Jails, Hospitals & Hip-Hop (2000)',
 'They Bite (1996)',
 'Black Tights (Les Collants Noirs) (1960)',
 'Identification of a Woman (Identificazione di una donna) (1982)']

In [29]:
# Recommendation based on past watched movies, and this time two movies is there in the history.
recommender.recommend_on_history()

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['12 Angry Men (1957)',
 'Not One Less (Yi ge dou bu neng shao) (1999)',
 'Cotton Mary (1999)',
 'Raining Stones (1993)',
 'Beloved/Friend (Amigo/Amado) (1999)']

In [30]:
# Recommendation based on this movie
recommender.recommend_on_movie('Dracula')

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Nemesis 2',
 'Best of the Best 3',
 'Mighty Morphin Power Rangers',
 'Gumby',
 'Die Hard']

In [31]:
# Recommendation based on past watched movies, and this time three movies is there in the history.
recommender.recommend_on_history()

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Run of the Country, The (1995)',
 "It's My Party (1995)",
 'Windows (1980)',
 'Night Porter, The (Il Portiere di notte) (1974)',
 'Forbidden Christ, The (Cristo proibito, Il) (1950)']

In [32]:
# Recommendation based on this movie
recommender.recommend_on_movie('Money Train (1995)')

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Shadow, The (1994)',
 'Black Mask (Hak hap) (1996)',
 'Stranger, The (1994)',
 'Shanghai Noon (2000)',
 'Thunderball (1965)']

In [33]:
# Recommendation based on past watched movies, and this time four movies is there in the history.
recommender.recommend_on_history()

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Men With Guns (1997)',
 'Tokyo Fist (1995)',
 'Rocky IV (1985)',
 "In God's Hands (1998)",
 "Player's Club, The (1998)"]

In [34]:
# Recommendation based on this movie
recommender.recommend_on_movie('GoldenEye (1995)')

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Anaconda (1997)',
 'Clear and Present Danger (1994)',
 'Surviving the Game (1994)',
 'Chain Reaction (1996)',
 'Rock, The (1996)']

In [35]:
# Recommendation based on past watched movies, and this time five movies is there in the history.
recommender.recommend_on_history()

C:\Users\Lenovo\Downloads\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Runaway Train (1985)',
 "Smilla's Sense of Snow (1997)",
 'Surviving the Game (1994)',
 'Anaconda (1997)',
 'Perfect Storm, The (2000)']